# Asset Analysis
Developed by Steve Kuhlman

In [1]:
using WGLMakie, AlgebraOfGraphics,  XLSX, DataFrames, Dates

In [4]:
function readMarket(marketFile)
  mdf=DataFrame(XLSX.readtable(marketFile, 1)...);
  mdf.Close=convert(Vector{Float64}, mdf.Close);    
  mdf
end

stripXLSX(s)=split(s, ".")[1]

function readMarkets(dataDir)
  marketFiles=readdir(dataDir)    
  marketPaths=map(x -> joinpath(dataDir, x), marketFiles)
  markets = map(readMarket, marketPaths) 
  marketArr = map(readMarket,  marketPaths)
  markets=reduce(vcat, marketArr)
  marketNames=map(stripXLSX, marketFiles)
  (markets, marketNames)
end

readMarkets (generic function with 1 method)

In [5]:
#Note: Enter the folder containing data here
dataDir="../data/market"
(markets, marketNames)=readMarkets(dataDir);

In [8]:
"Calculate the minumum closing value for each equity"
function getMinClose(gmarkets)
  closeHL=combine(gmarkets, :Close => minimum)
  minClose=Dict{String, Float64}()
  [minClose[closeHL[r, :Ticker]] = closeHL[r, :Close_minimum]  for r in 1:size(closeHL, 1)]
  minClose
end

function normalizeByStartPrice(gmarkets)
  
end

"Get the first and last date for the charts"
function getDateRange(market)
  #TODO get actual first and last, don't assume positions
  first=market[1,:timestamp]
  last=market[end,:timestamp]
  dateRange=first : Day(1) : last
end

function plotTicker(markets, marketNames, pos)
  dateRange=getDateRange(markets[pos])
  lines!(days, markets[pos].Close, label=marketNames[pos])
end

plotTicker (generic function with 1 method)

In [19]:
closeHL=combine(gmarkets, :Close => minimum)
minClose=Dict{String, Float64}()
[minClose[closeHL[r, :Ticker]] = closeHL[r, :Close_minimum]  for r in 1:size(closeHL, 1)]
minClose

Dict{String, Float64} with 5 entries:
  "JPM"  => 113.43
  "MRNA" => 109.18
  "AMD"  => 73.09
  "INTC" => 37.34
  "TM"   => 140.41

In [14]:
gmarkets=groupby(markets, :Ticker);

,timestamp,Open,High,Low,Close,AdjClose,Volume,Ticker
,Any,Any,Any,Any,Float64,Any,Any,Any
1,2021-01-04,92.11,96.06,90.92,92.3,92.3,5.18026e7,AMD
2,2021-01-05,92.1,93.21,91.41,92.77,92.77,3.4208e7,AMD
3,2021-01-06,91.62,92.28,89.46,90.33,90.33,5.19117e7,AMD
4,2021-01-07,91.33,95.51,91.2,95.16,95.16,4.28972e7,AMD
5,2021-01-08,95.98,96.4,93.27,94.58,94.58,3.98164e7,AMD
6,2021-01-11,94.03,99.23,93.76,97.25,97.25,4.86002e7,AMD
7,2021-01-12,97.86,98.97,94.07,95.36,95.36,6.76723e7,AMD
8,2021-01-13,93.01,93.89,90.84,91.78,91.78,6.07129e7,AMD
9,2021-01-14,91.77,92.36,90.45,90.79,90.79,5.34082e7,AMD


In [15]:
#describe(get(gmarkets, ("AMD",), nothing))
#marketNames
#6 / minClose["AMD"]
#map(markets.CloseNorm .= markets.Close ./ minClose[markets.Ticker]

LoadError: UndefVarError: minClose not defined

In [13]:
getMinClose(gmarkets)

LoadError: ArgumentError: column name :Ticker not found in the data frame

In [ ]:
dateRange=getDateRange(markets)
gmarkets=groupby(markets, :Ticker);
minClose=getMinClose(gmarkets)
plotCount=[x for x in 1:size(marketNames,1)]
gmarkets=groupby(markets, :Ticker);

plt=data(markets) * mapping(:timestamp, :Close, color=:Ticker) * visual(Lines)
draw(plt)


In [ ]:
markets